# Data Science Capstone - Predicting Car Accident Severity
**Sergio Aguirre**
***

## **Introduction - Business Problem**

Daily we are faced with making decisions that, however simple they may seem, can change our lives. One of these decisions is which route to take to reach a specific destination.
Generally, the optimization and decision go through which route is shorter, will take less time or is less congested, but what about security?.

The goal of this work is to analyze data in order to provide a model that will predict the severity of an accident, given a number of features that will be extracted from the car collision data from Seattle. The model will classify and predict severity, so a driver can use this model through a third-party or government app to change course or drive more carefully, given that he is not wanting to get involved into an accident.

This project will use Machine Learning to build models to predict the severity of the car accidents according to the external factors in the car collision data of the city of Seattle. This model can be applied to multiple practical conditions, such as route planning, signals placement or road improvements. This model can be methodologically extrapolated to other cities anywhere in the world that have similar data collection systems.

This project is aimed at government agencies and private companies that want to optimize their route planning and prevent accidents.

## Data

The car collision data that will be used comes from the Seattle Department of Transportation, which records the information related to traffic accidents and the associated factors. This dataset includes a severity indicator (SEVERITYCODE) for each traffic accident where the values are 1 for minors and 2 for severe accidents. This indicator will be the target variable of this project.
The data also includes external information associated with accidents that is of direct utility for the study, such as weather, road and lighting conditions. Obviously crucial information about the date and time of occurrence, as well as internal factors such as the influence of alcohol (UNDERINFL), speeding (SPEEDING) or inattention that allow us to rule out certain accidents from our data that do not have to do purely with external factors. For a complete description of each parameter, consult the metadata included in the repository.

The data looks like the following:

![](Images/Dataframe1.png?raw=true "")

The focus of the project relies on external variables that are not in driver's control to predict the severity of the accident. Therefore, when reviewing the different columns of the data, it is possible to eliminate some that are not of our interest, these are:

|COLUMN NAMES	|REASON FOR DROPING 	| 
|:-:	|:-:	|
|OBJECTID, STATUS, REPORTNO,<br> EXCEPTRSNCODE, EXCEPTRSNDESC 	| These are mostly identification codes for each particular accident<br> and do not add valuable information.|
| INCKEY, INTKEY, CROSSWALKKEY, SEGLANEKEY 	|  These are identification codes for intersections, pedestrian crossing and lane that <br> will be considered included in the location information and coordinates provided.	| 
|COLDETKEY, SDOTCOLNUM, SDOT_COLDESC,<br> ST_COLDESC, SDOT_COLCODE, ST_COLCODE 	| These are specific codes for types of accidents, which will not be used since <br>we want to predict the severity without knowing the type of accident.	| 
|PERSONCOUNT, PEDCOUNT, PEDCYLCOUNT,<br> VEHCOUNT,PEDROWNOTGRNT, HITPARKEDCAR  	| These columns correspond to the number of people, vehicles and others <br>associated with each accident. Again, as we will not have this type of<br> information as input, since we are located before an accident, these will not be used. 	| 
|SEVERITYCODE.1, INCDATE, SEVERITYDESC,<br> JUNCTIONTYPE, LOCATION| The information in these columns is contained in others. <br>This is SEVERITYCODE.1 and SEVERITYDESC in SEVERITYCODE, <br>INCDATE in INCDTTM, JUNCTIONTYPE in ADDRTYPE <br>and LOCATION in X,Y Coordinates.|

After droping the columns the data looks like the following:

![](Images/Dataframe2.png?raw=true "")

At check the type of content of each column it was found that in some cases of columns that contain a yes or no they had more types of answers. Therefore, the content was standardized as follows, considering that it is preferred to have the variables numerically:

|COLUMN NAME	|CONTENT UNIQUE 	|  REPLACEMENT          |
|:-:	|:-:	|  :-:  | 
|INATTENTIONIND	| [nan 'Y']	|  nan &rarr; 0 y Y &rarr; 1  |
|SPEEDING	    | [nan 'Y']	|  nan &rarr; 0 y Y &rarr; 1  |
|UNDERINFL	    | ['N' '0' nan '1' 'Y']	|  N &rarr; 0 , Y &rarr; 1 |

NaN was considered to contain both negative and unknown variables for the INATTENTIONIND and SPEEDING columns. On the other hand, for the UNDERINFL column, it was decided to simply drop the rows with unknown data (NaN) since negative and affirmative cases are clearly defined.

In the same way, since the data could present other rows with unknown information. The total data was reviewed in search of unknown data (NaN) and the rows were dropped away. The summary of unknown data by column is as follows:

||SEVERITYCODE | X|Y|ADDRTYPE|COLLISIONTYPE|INCDTTM|INATTENTIONIND|UNDERINFL|WEATHER|ROADCOND|LIGHTCOND|SPEEDING| 
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|<b> NaN COUNT  |0|5334| 5334|1926|4904|  0|  0|  0|5081|5012|5170|0|




Now with this little data pre-processing we proceed to take a little look at the data of Collision Type:

![](Images/exploratory_01_CT.png?raw=false "")

These graphs immediately indicate that most traffic accidents do not occur in conditions as adverse as one would first believe. In fact, they mostly occur on days with clear weather, dry roads in good condition, and in broad daylight. Therefore, a good performance of the model could no longer be expected given these conditions. Also, the time, path, and brightness typing tends to be too broad for the infrequency of some types. Therefore, later on we will proceed to unify some criteria for simplicity.

In the target category, Severity Code, it is observed that the severity of the accident tends to be greater with the involvement of pedestrians, cyclists, and in rear-end collisions and angle collisions.


Next,it will be preented a preview of the effect of the date and time on the distribution of collisions and also on the driver's condition variables:

![](Images/exploratory_02_DaT.png?raw=true "")

In the first graph it can be seen that most accidents occur in the summer months, which obviously coincides with the better climates. This reinforces what was seen in the previous visualization. This behavior could be explained by an increase in drivers with less experience, i.e., teenage drivers who have more free time. Increase in construction works on those dates, excessive heat that fatigue the driver and also affecting the maintenance conditions of the vehicle.

The second plot shows that most of the accidents are not related to the conditions or state of the driver. However, among the conditions that have been reviewed, the one that has the most impact corresponds to collisions due to driver inattention. In addition, it is a fact that the idea that Fridays are the days with the most accidents regardless of the driver's condition.

In the third plot it is reinforced that Fridays are complex and the ones with the highest number of collisions in all months of the year. A separate point is the curious fact that Fridays in October are considerably the most accident-prone days. In relation to other months, Wednesdays and Saturdays in October are not far behind and also have a high frequency of accidents. It is an interesting point to investigate what happens in October in Seattle, if there is any event that may be related to these high rates.

Finally, the last graph shows us that accidents are more prone in peak hours. This is in the morning or late afternoon. An anomalous behavior can be detected in relation to midnight, it is logical that it appears with a high frequency of accidents but the difference in relation to other hours should not be so abysmal. Therefore, a revision flag is raised for the time of day data.

